In [1]:
# Include project root in path
# import sys
# sys.path.append("../..")

# Load data
from src.common import preprocessing, datasets
data = datasets.read_gammas()

# Preprocess data
from src.common.preprocessing import preprocess
from src.common import PARAMS_HILLAS, PARAMS_TRUE_SHOWER, PARAMS_CLEAN_IMAGE_M1, PARAMS_CLEAN_IMAGE_M2
train, validation, test = preprocess(data, normalize_params=PARAMS_HILLAS + PARAMS_TRUE_SHOWER + PARAMS_CLEAN_IMAGE_M1 + PARAMS_CLEAN_IMAGE_M2)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from src.common.HexaToParallelogram import HexaToParallelogram


# class Combine(nn.Module):
#     def __init__(self):
#         self.cnn:nn.Module = None
#         self.input2size

#         self.fc1

#     def forward(self, images, linear):
#         X = fc1(torch.cat(self.cnn(images), linear))


class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(2, 6, 5)  # 22x22
        self.pool = nn.MaxPool2d(2, 2)  # 18x18
        self.conv2 = nn.Conv2d(6, 16, 5)  # 9x9


    def forward(self, images):
        x = self.pool(F.relu(self.conv1(images)))
        x = self.pool(F.relu(self.conv2(x)))
        return x


class ParticleCNNRegressor(nn.Module):
    def __init__(self, num_additional_parameters):
        super().__init__()

        self.hex2par = HexaToParallelogram()
        self.cnn = CNN()

        self.fc1 = nn.Linear(16 * 6 * 6 + num_additional_parameters, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 8)

    def forward(self, images, additional_parameters):
        x = torch.stack([self.hex2par(images[0]), self.hex2par(images[1])])

        x = self.cnn(x)
        x = torch.flatten(x)

        x = torch.cat((x, additional_parameters))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x


In [6]:
import torch.optim as optim

regressor = ParticleCNNRegressor(len(PARAMS_HILLAS))
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(regressor.parameters(recurse=True), lr=0.00001, momentum=0.0)

for epoch in range(2):
    running_loss = 0.0

    for i in range(len(train)):
        d = train.iloc[i]
        optimizer.zero_grad()

        clean_image_m1 = torch.tensor(d[PARAMS_CLEAN_IMAGE_M1].values, dtype=torch.float)
        clean_image_m2 = torch.tensor(d[PARAMS_CLEAN_IMAGE_M2].values, dtype=torch.float)
        images = torch.stack([clean_image_m1, clean_image_m2])
        additional_properties = torch.tensor(d[PARAMS_HILLAS].values, dtype=torch.float)
        target_values = torch.tensor(d[PARAMS_TRUE_SHOWER].values, dtype=torch.float)


        outputs = regressor(images, additional_properties)
        loss = criterion(outputs, target_values)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # print(loss.item())
        if i % 200 == 199:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0

        


    
    

[1,   200] loss: 0.492
[1,   400] loss: 0.172
[1,   600] loss: 1.928
[1,   800] loss: -0.154
[1,  1000] loss: 0.062
[1,  1200] loss: 0.311
[1,  1400] loss: 0.602
[1,  1600] loss: 1.363
[1,  1800] loss: 0.332
[1,  2000] loss: 0.139
[1,  2200] loss: -0.150
[1,  2400] loss: -0.650
[1,  2600] loss: -0.771
[1,  2800] loss: -0.025
[1,  3000] loss: -0.203
[1,  3200] loss: 1.072
[1,  3400] loss: -0.258
[1,  3600] loss: -0.302
[1,  3800] loss: -0.562
[1,  4000] loss: -0.193
[1,  4200] loss: 0.769
[1,  4400] loss: -0.437
[1,  4600] loss: 0.274
[1,  4800] loss: -0.537
[1,  5000] loss: 0.531
[1,  5200] loss: 0.882
[1,  5400] loss: 0.650
[1,  5600] loss: -0.316
[1,  5800] loss: 0.787
[1,  6000] loss: -0.116
[1,  6200] loss: 0.533
[1,  6400] loss: -0.908


KeyboardInterrupt: 